# Introduction

Notebook para análise de experimentos com Single Task Networks treinadas usando a técnica de Transfer Learning.

A rede de base é sempre uma VGG16 com os pesos da ImageNet.

Um total de 23 redes são treinadas, uma para cada requisito ICAO. Cada requisito tem apenas duas classes possíveis: COMPLIANT ou NÃO-COMPLIANT, sendo assim uma classificação binária para um rede Single Task Network (STL), já para uma rede Multi Task Network (MTL) sáo 23 saídas simultâneas com dois neurônios cada.

Os dados de preprocessamento, treinamento e avaliação de cada rede está no [Neptune](https://ui.neptune.ai/guilhermemg/icao-nets-training/experiments?viewId=6bb9c9c5-9198-432c-a8d7-fca327e76dcf).

As redes foram baixadas para pastas locais usando o notebook download_models.ipynb.

Os resultados obtidos são detalhados abaixo.

# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import neptune.new as neptune
import pandas as pd

if '..' not in sys.path:
    sys.path.insert(0, '..')

from src.configs import config
from src.m_utils import constants as cts

# General Results - Single Task Networks

## Utilitary Functions

In [3]:
def process_gt_names(gt_names):
    gt_names_dict = {'train_validation_test': gt_names.split(':')[-1]}
    train_val_test = gt_names_dict['train_validation_test']
    train_val_test = train_val_test[:-3]
    train_val_test = train_val_test[3:]
    return train_val_test


def get_experiments_metrics(exp_id):
    print(f'Getting Experiment {exp_id} metrics')
    try:
        neptune_proj = None
        neptune_proj = neptune.init(project=config.NEPTUNE_PROJECT, 
                                    api_token=config.NEPTUNE_API_TOKEN,
                                    run=exp_id)
        
        ds = neptune_proj['properties/gt_names'].fetch()
        alg = neptune_proj['properties/aligned'].fetch()
        req = neptune_proj['properties/icao_reqs'].fetch()
        is_mtl = bool(neptune_proj['properties/is_mtl_model'].fetch())

        ds = process_gt_names(ds)
        aligned = 'aligned' if bool(alg) == True else 'not_aligned'
        
        metrics = None
        if not is_mtl:
            req = req[2:-2]
            
            if req == 'ink_mark':
                metrics = {
                    'exp_id': neptune_proj['sys/id'].fetch(),
                    'req': req,
                    'aligned': aligned,
                    'eer_mean_valid': '',
                    'eer_interp_valid': '',
                    'eer_mean_test': '',
                    'eer_interp_test': ''
                }
            else:
                metrics = {
                    'exp_id': neptune_proj['sys/id'].fetch(),
                    'req': req,
                    'aligned': aligned,
                    'eer_mean_valid': float(neptune_proj['metrics/validation/EER_mean'].fetch()),
                    'eer_interp_valid': float(neptune_proj['metrics/validation/EER_interp'].fetch()),
                    'eer_mean_test': float(neptune_proj['metrics/test/EER_mean'].fetch()),
                    'eer_interp_test': float(neptune_proj['metrics/test/EER_interp'].fetch())
                }
            
            return metrics
        
        else:
            reqs_metrics = {}
            for r in list(cts.ICAO_REQ):
                r = r.value
                print(f'  requisite: {r}')
                
                if r == 'ink_mark':
                    reqs_metrics[r] = {
                        'exp_id': neptune_proj['sys/id'].fetch(),
                        'req': r,
                        'aligned': aligned,
                        'eer_mean_valid': '',
                        'eer_interp_valid': '',
                        'eer_mean_test': '',
                        'eer_interp_test': ''
                    }
                else:
                    reqs_metrics[r] = {
                        'exp_id': neptune_proj['sys/id'].fetch(),
                        'req': r,
                        'aligned': aligned,
                        'eer_mean_valid': float(neptune_proj[f'metrics/validation/{r}/EER_mean'].fetch()),
                        'eer_interp_valid': float(neptune_proj[f'metrics/validation/{r}/EER_interp'].fetch()),
                        'eer_mean_test': float(neptune_proj[f'metrics/test/{r}/EER_mean'].fetch()),
                        'eer_interp_test': float(neptune_proj[f'metrics/test/{r}/EER_interp'].fetch())
                    }
            
            return reqs_metrics
        
    except Exception as e:
        raise e
    
    finally:
        if neptune_proj is not None:
            neptune_proj.stop()

## Final Dataframe COLUMNS variable

In [4]:
COLUMNS = ['exp_id', 'req', 'aligned', 'eer_mean_valid', 'eer_interp_valid', 'eer_mean_test', 'eer_interp_test']

## Download STL Experiments Data

In [7]:
exp_data = pd.read_csv('single_task_exps_data/icao-nets-training-2.csv')

final_df = pd.DataFrame()

for idx,exp_id in enumerate(exp_data.Id):
    metrics_dict = get_experiments_metrics(exp_id)
    final_df = final_df.append(metrics_dict, ignore_index=True)
    
display(final_df.head(23))

print(final_df.shape)b

Getting Experiment ICAO-186 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-186
Getting Experiment ICAO-185 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-185
Getting Experiment ICAO-184 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-184
Getting Experiment ICAO-183 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-183
Getting Experiment ICAO-182 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-182
Getting Experiment ICAO-181 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-181
Getting Experiment ICAO-180 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-180
Getting Experiment ICAO-179 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-179
Getting Experiment ICAO-178 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-178
Getting Experiment ICAO-177 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-tr

,aligned,eer_interp_test,eer_interp_valid,eer_mean_test,eer_mean_valid,exp_id,req
0,not_aligned,0.1636,0.2273,0.144,0.2292,ICAO-186,red_eyes
1,not_aligned,0.0746,0.0901,0.073,0.0912,ICAO-185,background
2,not_aligned,0.1387,0.1936,0.1294,0.1894,ICAO-184,hair_eyes
3,not_aligned,0.3282,0.2907,0.3179,0.2905,ICAO-183,pixelation
4,not_aligned,0.0211,0.1268,0.0106,0.0634,ICAO-182,washed_out
5,not_aligned,0.2139,0.2094,0.2113,0.2102,ICAO-181,skin_tone
6,not_aligned,,,,,ICAO-180,ink_mark
7,not_aligned,0.1186,0.0893,0.1021,0.0796,ICAO-179,blurred
8,not_aligned,0.0971,0.099,0.0918,0.0988,ICAO-178,sh_head
9,not_aligned,0.1481,0.155,0.1518,0.1531,ICAO-177,sh_face


(23, 7)


### Record Metrics STL

In [11]:
final_df = final_df[COLUMNS]

display(final_df.head())

final_df.to_csv('single_task_exps_data/STL_metrics.csv', index=False)

,exp_id,req,aligned,eer_mean_valid,eer_interp_valid,eer_mean_test,eer_interp_test
0,ICAO-186,red_eyes,not_aligned,0.2292,0.2273,0.144,0.1636
1,ICAO-185,background,not_aligned,0.0912,0.0901,0.073,0.0746
2,ICAO-184,hair_eyes,not_aligned,0.1894,0.1936,0.1294,0.1387
3,ICAO-183,pixelation,not_aligned,0.2905,0.2907,0.3179,0.3282
4,ICAO-182,washed_out,not_aligned,0.0634,0.1268,0.0106,0.0211


# Handcrafted 1

## Download MTL Experiments Data - 10 epochs

In [6]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-199')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_1_10_epochs.csv', index=False)

Getting Experiment ICAO-199 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-199
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


## Download MTL Experiments Data - 200 epochs

In [7]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-192')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_1_200_epochs.csv', index=False)

Getting Experiment ICAO-192 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-192
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


## Download MTL Experiment Data - Fine-Tuned - 200 epochs

In [5]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-204')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_fine_tuned_200_epochs.csv', index=False)

Getting Experiment ICAO-204 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-204
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


# Handcrafted 2

## Download MTL Experiment Data - 50 epochs

With Rotation and width_shift_range = 0.2 and height_shift_range = 0.1

In [5]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-228')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_2_50_epochs.csv', index=False)

Getting Experiment ICAO-228 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-228
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


## Download MTL Experiment Data - 50 epochs

No Rotation in Data Augmentation

In [7]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-239')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_2_50_epochs_no_rotation.csv', index=False)

Getting Experiment ICAO-239 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-239
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


## Download MTL Experiment Data - 200 epochs

No Rotation in Data Augmentation

In [6]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-238')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_2_200_epochs_no_rotation.csv', index=False)

Getting Experiment ICAO-238 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-238
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


## Download MTL Experiment Data - 50 epochs

No Rotation and minor shift (width and height) in Data Augmentation

In [4]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-242')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_2_50_epochs_no_rotation_minor_shift.csv', index=False)

Getting Experiment ICAO-242 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-242
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


## Download MTL Experiment Data - 200 epochs

In [5]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-230')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_2_200_epochs.csv', index=False)

Getting Experiment ICAO-230 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-230
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes


# Handcrafted 3

## Download MTL Experiment Data - 10 epochs

In [5]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-336')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_3_10_epochs_NEW.csv', index=False)

Getting Experiment ICAO-336 metrics
https://app.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-336


Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/you-should-know/what-can-you-log-and-display#hardware-consumption


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes
Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.


All 5 operations synced, thanks for waiting!


/tmp/ipykernel_82693/3368960707.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/3368960707.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/3368960707.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/3368960707.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/3368960707.py:5: FutureWarning: The frame.append method is deprecat

## Download MTL Experiment Data - 50 epochs

In [6]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-337')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_3_50_epochs_NEW.csv', index=False)

Getting Experiment ICAO-337 metrics
https://app.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-337
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes
Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.


All 5 operations synced, thanks for waiting!


/tmp/ipykernel_82693/245912713.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/245912713.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/245912713.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/245912713.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/245912713.py:5: FutureWarning: The frame.append method is deprecated an

## Download MTL Experiment Data - 200 epochs

In [7]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-338')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/MTL_metrics_handcrafted_3_200_epochs_NEW.csv', index=False)

Getting Experiment ICAO-338 metrics
https://app.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-338
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes
Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.


All 1 operations synced, thanks for waiting!


/tmp/ipykernel_82693/2080583285.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/2080583285.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/2080583285.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/2080583285.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(metrics, ignore_index=True)
/tmp/ipykernel_82693/2080583285.py:5: FutureWarning: The frame.append method is deprecat

# NAS Approach 1 - Random

In [5]:
final_df = pd.DataFrame()

reqs_metrics = get_experiments_metrics('ICAO-273')
for req,metrics in reqs_metrics.items():
    final_df = final_df.append(metrics, ignore_index=True)
    
final_df = final_df[COLUMNS]
final_df.to_csv('multi_task_exps_data/NAS_metrics_approach_1_50_epochs.csv', index=False)

Getting Experiment ICAO-273 metrics
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-273
  requisite: mouth
  requisite: rotation
  requisite: l_away
  requisite: eyes_closed
  requisite: close
  requisite: hat
  requisite: dark_glasses
  requisite: frames_heavy
  requisite: frame_eyes
  requisite: flash_lenses
  requisite: veil
  requisite: reflection
  requisite: light
  requisite: sh_face
  requisite: sh_head
  requisite: blurred
  requisite: ink_mark
  requisite: skin_tone
  requisite: washed_out
  requisite: pixelation
  requisite: hair_eyes
  requisite: background
  requisite: red_eyes
